In [1]:
import pandas as pd
import MeCab
from gensim import models

df = pd.read_csv("../data/tweets.csv")
mecab = MeCab.Tagger ("-Owakati")
print(mecab.parse("pythonが大好きです"))

df = df[pd.isnull(df.retweeted_status_id)]
df = df[pd.isnull(df.in_reply_to_status_id)]
df = df[df.text.str.find("http")==-1]

for index, row in df.iterrows():
    if not pd.isnull(row.text):
        df.set_value(index, "text", mecab.parse(row["text"]))        
        
for index, row in df.iterrows():
    if not pd.isnull(row.text):
        print(row.text)
    if index > 10 : break;

python が 大好き です 

無理 だ ー 

人間 よ 、 24 に 、 分類 さ れよ ！ 

計算 中 ・ ・ ・ 

えい 、 ` from scipy . cluster . vq import vq , kmeans , whiten ` 

k - means やる ぞ 

お は よー ！ 6 h 28 m ね た よ ！ January 27 , 2016 at 01 : 19 AM → January 27 , 2016 at 08 : 06 AM 

ねむい ー 

Doc 2 Vec の 基底 ベクトル は Word 2 Vec の それ と 一緒 っぽい な 



In [2]:
df.to_csv("../data/formatted_tweets.csv")

In [3]:
import itertools
class LabeledLineSentence(object):
    def __init__(self, df):
        self.df = df

    def __iter__(self):
        for index, row in self.df.iterrows():            
            yield models.doc2vec.LabeledSentence(row["text"].split(), ['text_%s' % index])
            

sentence = LabeledLineSentence(df)
print(list(sentence)[:5])
len(df)

[LabeledSentence(words=['無理', 'だ', 'ー'], tags=['text_1']), LabeledSentence(words=['人間', 'よ', '、', '24', 'に', '、', '分類', 'さ', 'れよ', '！'], tags=['text_2']), LabeledSentence(words=['計算', '中', '・', '・', '・'], tags=['text_3']), LabeledSentence(words=['えい', '、', '`', 'from', 'scipy', '.', 'cluster', '.', 'vq', 'import', 'vq', ',', 'kmeans', ',', 'whiten', '`'], tags=['text_4']), LabeledSentence(words=['k', '-', 'means', 'やる', 'ぞ'], tags=['text_5'])]


8423

In [4]:
from gensim import models
model = models.Doc2Vec()
model.build_vocab(sentence)

for _ in range(10):
    model.train(sentence)
    print("トレーニング" + str(_) + "終了")

トレーニング0終了
トレーニング1終了
トレーニング2終了
トレーニング3終了
トレーニング4終了
トレーニング5終了
トレーニング6終了
トレーニング7終了
トレーニング8終了
トレーニング9終了


In [5]:
# model.vocab.keys()

In [6]:
model.most_similar("リクルート")

[('普通', 0.6398686170578003),
 ('波', 0.636785626411438),
 ('iCloud', 0.6355605721473694),
 ('秋', 0.6327695846557617),
 ('違い', 0.6275097131729126),
 ('標準', 0.61744225025177),
 ('最後', 0.6063286066055298),
 ('スクショ', 0.599561333656311),
 ('うち', 0.5984364748001099),
 ('ipad', 0.5951722860336304)]

In [7]:
base = "text_1"
key, index = base.split("_")
print(df.ix[int(index)][key])
for offset, similality in model.docvecs.most_similar(base):
    doctag = model.docvecs.offset2doctag[offset]
    key, index = doctag.split("_")
    print("%s%%"%int(similality*100),df.ix[int(index)][key])

無理 だ ー 

81% 無料 で 遊ん だ ー 

75% NSTimer が 動か ない 。 なんで だ ー 。 

70% だ だ し 、 Template 系 が 入ら ない なぁ 。 

69% bitbucket の Slack へ の 連携 が びみょ ー だ 

68% 迷惑 メール が 来る よう に なっ た 

67% ちょ ー いい ランチ だっ た ！ たのし ー 

66% また 、 上 、 ネット つながら ない やつ だ … . 

66% どうやら 、 ペンギン ちゃん が 今夜 リリース さ れる みたい だ 

66% 駄目 だ camphchat が 動か ない 。 なんで だ 

66% すごい シングル トン だ 



In [8]:
from scipy.cluster.vq import vq, kmeans, whiten
centroid, destortion = kmeans(model.docvecs, 10 , iter=100, thresh=1e-05)

In [9]:
labels, dist = vq(model.docvecs, centroid)
print(labels[:5]) # どのクラスターに入ってるか
print(dist[:5]) # 中心からの距離

[5 7 4 1 5]
[ 0.28320223  0.34859508  0.2048361   0.29068244  0.28006327]


In [10]:
df = pd.read_csv("../data/tweets.csv")
for label in range(10):
    print(label,"スタート")
    print_i = 0
    for i in range(model.docvecs.count):
        if labels[i] == label:
            dist_vec = vq([model.docvecs[i]], centroid)
            doctag = model.docvecs.offset2doctag[i]
            key, index = doctag.split("_")
            if int((1.-dist[i])*100) and not pd.isnull(df.ix[int(index)][key]):
#                 print(dist_vec)
                print("%s%%"%int((1.-dist[i])*100),df.ix[int(index)][key].strip("\t\n"))
                print_i +=1
                if print_i > 3 : 
                    print("\n");break;

0 スタート
52% ねむいー
63% Doc2Vecの基底ベクトルはWord2Vecのそれと一緒っぽいな
66% POSTDのパーカーはポッドキャストに出ればもらえるらしい
78% やこちゃんの趣味w


1 スタート
70% えい、 `from scipy.cluster.vq import vq, kmeans, whiten`
82% えっーっと、やこです！！！
81% タイガーナッツ気になる
73% みんな電子タバコにしてくんないかな


2 スタート
76% ぶっちゃけ感w
73% 日経平均さん！
70% 企業のTechBlogで成功しているところってどこやろうなぁ。
72% 機械学習で質問できるところないだろうかぁ


3 スタート
77% フェスや！フェス！
69% どういう人をとりたいかでテーマ変わってきそうだから、個人で分散させて、有名人を複数人作るほうが良さそうな気はしている
66% 個人でやってるブログが良いことが多いことが多い気がする。それで会社に興味持つ流れのほうが自然だわなぁ、って気はする。
69% Theanoはお勉強にはいいけど、めんどくさい。しばらくChainerかなっという感。


4 スタート
79% 計算中・・・
75% 聴き終わった
76% フェスは負けまくってる
73% 大麦にはグルテンが入ってない！


5 スタート
71% 無理だー
71% k-meansやるぞ
72% softmaxを使えば0にはならないからクロスエントロピー大丈夫なのか
77% この前見れなかったから楽しみね


6 スタート
62% おはよー！ 6h 28m ねたよ！ January 27, 2016 at 01:19AM →January 27, 2016 at 08:06AM
69% おはよー！ 5h 57m ねたよ！ January 26, 2016 at 01:34AM →January 26, 2016 at 07:46AM
70% おはよー！ 8h 31m ねたよ！ January 25, 2016 at 01:09AM →January 25, 2016 at 09:52AM
69% おはよー！ 6h 49m ねたよ！ January 23, 2016 at 02:25AM →January 23, 2016 at 09:37AM

In [11]:
import numpy as np
sim_matrix = []
size = model.docvecs.count
for i in range(model.docvecs.count):
    sim_vec = np.zeros(size)
    for word_index, sim_val in model.docvecs.most_similar(i, topn=100):
        sim_vec[word_index] = sim_val
    sim_matrix.append(sim_vec)

In [12]:
from sklearn.decomposition import TruncatedSVD
dimension = 100
lsa = TruncatedSVD(dimension)
info_matrix = lsa.fit_transform(sim_matrix)

In [13]:
centroid, destortion = kmeans(info_matrix, 10 , iter=100, thresh=1e-05)

In [14]:
labels, dist = vq(info_matrix, centroid)
print(labels[:5]) # どのクラスターに入ってるか
print(dist[:5]) # 中心からの距離

[9 1 6 5 9]
[ 3.56423178  3.75280822  2.52135968  1.90380796  3.18993187]


In [15]:
df = pd.read_csv("../data/tweets.csv")
for label in range(10):
    print(label,"スタート")
    print_i = 0
    for i in range(model.docvecs.count):
        if labels[i] == label:
            doctag = model.docvecs.offset2doctag[i]
            key, index = doctag.split("_")
            if int((1.-dist[i])*100) and not pd.isnull(df.ix[int(index)][key]):
#                 print(dist_vec)
                print("%s%%「"%int((1.-dist[i])*100),df.ix[int(index)][key].strip("\t\n"),"」")
                print_i +=1
                if print_i > 4 : break;
    print("\n");

0 スタート
-113%「 個人でやってるブログが良いことが多いことが多い気がする。それで会社に興味持つ流れのほうが自然だわなぁ、って気はする。 」
-159%「 という超ど素人の意見です 」
-170%「 Mantle to Realmがめんどくさい。結局書かなあかんのか 」
-192%「 なんでや 」
-260%「 日本のVCでそういうところまでサポートしてくれるところってあるのかな？

あんまり聞かない気がするけど 」


1 スタート
-275%「 人間よ、24に、分類されよ！ 」
-277%「 フェス、カンペキカラダ、勝った！！ 」
-309%「 BASEで、ケーキ食べて、日本酒飲んで、マジゥクザギャザリングやって、カルカソンヌした 」
-277%「 中目黒付近に住んでる人多いのね。びっくりした。 」
-182%「 そういえば、僕、チャーシューが、食べれません 」


2 スタート
-131%「 おはよー！ 6h 28m ねたよ！ January 27, 2016 at 01:19AM →January 27, 2016 at 08:06AM 」
-222%「 おはよー！ 5h 57m ねたよ！ January 26, 2016 at 01:34AM →January 26, 2016 at 07:46AM 」
-164%「 おはよー！ 6h 49m ねたよ！ January 23, 2016 at 02:25AM →January 23, 2016 at 09:37AM 」
-108%「 おはよー！ 7h 53m ねたよ！ January 22, 2016 at 01:10AM →January 22, 2016 at 09:21AM 」
-40%「 おはよー！ 5h 58m ねたよ！ January 21, 2016 at 01:38AM →January 21, 2016 at 07:52AM 」


3 スタート
-196%「 体をアルカリ性に変えてこ 」
-237%「 2こ同じビルド番号であがってもうた 」
-144%「 クライアント側実装していて、サーバー開発の時に、swagger入れておいて正解だったと思ってる 」
-160%「 集中力を鍛える 」
-147%「 ダイオウイカ対スーパーショット 」


4 スタート
-156%「 鍵って、鍵番